### Given a topic research the latest news

In [28]:
import sys
sys.path.append('..')

from dotenv import load_dotenv
load_dotenv()

from utils import utils , llms


In [29]:
utils.set_prefix("../output/banksy")
scenes = utils.get_scenes()
# Extract lines starting with "[Scene"
scenes

[Scene(start_time=None, duration=None, scene_title='Banksy: An Overview and Recent Developments', description='Introduction to Banksy and his mysterious identity, along with a mention of recent developments.', content='Welcome back to our channel, folks! Today, we dive into the captivating world of Banksy, the elusive street artist and activist. Known for his satirical street art and subversive messages, Banksy has been making waves around the globe since the 1990s. Stick around as we explore his remarkable journey and the latest developments in his art.'),
 Scene(start_time=None, duration=None, scene_title="Banksy's Origin and Artistic Style", description="An exploration of Banksy's roots in the Bristol underground scene and his unique artistic style.", content="But first, let's go back to where it all began. Banksy's roots can be traced to the vibrant underground scene in Bristol, England. Collaborating with fellow artists and musicians, Banksy honed his skills and developed a distin

In [30]:

import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained("SG161222/Realistic_Vision_V2.0", torch_dtype=torch.float16)  

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [38]:
pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention()

In [39]:
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_ddim.DDIMScheduler,
 diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultiste

In [40]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [41]:
positive_prompt = """
(high detailed skin:1.2, high quality, reportage, journalism),
8k uhd, dslr, soft lighting, film grain, Fujifilm XT3
"""

negative_prompt="""
((text:1.5, deformed iris, deformed pupils, deformed hands, nsfw)),
((semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4)),
close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate,
morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation,
deformed, blurry, burns, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured,
gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs,
fused fingers, too many fingers, long neck
"""

In [34]:
import json
sentences_durations = json.loads(open(utils.VOICEOVER_SENTENCES_DURATIONS).read())
sentences_durations

[{'sentence': '"Ladies and gentlemen, welcome back to our exceptional channel!',
  'duration': 8},
 {'sentence': 'Today, we embark on a captivating journey into the enigmatic world of Banksy, the illustrious street artist and activist.',
  'duration': 15},
 {'sentence': 'Known far and wide for his satirical street art and subversive messages, Banksy has been causing quite the stir across the globe since the 1990s.',
  'duration': 14},
 {'sentence': 'So, stick around as we delve deep into his extraordinary voyage and uncover the latest developments in his artistic endeavors."',
  'duration': 14},
 {'sentence': '"But first, let us transport ourselves back to where it all began.',
  'duration': 7},
 {'sentence': "Banksy's roots can be traced to the vibrant underground scene in Bristol, England.",
  'duration': 9},
 {'sentence': 'Collaborating with fellow artists and musicians, Banksy honed his skills and crafted a distinctive stenciling technique.',
  'duration': 13},
 {'sentence': 'His a

In [35]:
## LLM To create thumbnail prompt from scenes
from langchain import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.chat_models import ChatOpenAI

template="""
You are a creative artist, who will generate prompts for a text-to-image model Stable Diffusion for the human.
who will provide you with a JSON containing an array of objects of "sentence"

Your task is to return the same json containing an additional "prompt" that describes an image that would
be most suitable for the corresponding "sentence".
Regardless of the sentence, make sure you stay on topic, and always remember the general context of the text provided
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template("{text}")

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [36]:
chain = LLMChain(llm=ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo"), prompt=chat_prompt)
prompts_raw = chain.run(json.dumps(sentences_durations))
prompts_raw

'[{"sentence": "\\"Ladies and gentlemen, welcome back to our exceptional channel!", "duration": 8, "prompt": "An image of a crowd of people eagerly awaiting the start of a captivating show."}, {"sentence": "Today, we embark on a captivating journey into the enigmatic world of Banksy, the illustrious street artist and activist.", "duration": 15, "prompt": "An image of a mysterious figure in a hoodie creating a thought-provoking mural on a city wall."}, {"sentence": "Known far and wide for his satirical street art and subversive messages, Banksy has been causing quite the stir across the globe since the 1990s.", "duration": 14, "prompt": "An image of a controversial Banksy artwork displayed prominently in a public space, attracting a crowd of onlookers."}, {"sentence": "So, stick around as we delve deep into his extraordinary voyage and uncover the latest developments in his artistic endeavors.\\"", "duration": 14, "prompt": "An image of a map with a path leading to various locations whe

In [42]:
guidance_scale        = 0.7
num_inference_steps   = 50
num_images_per_prompt = 3
image_height = 512
image_width = 768

for i,line in enumerate(json.loads(prompts_raw)):
    prompt = line["prompt"]
    images = pipeline(
        prompt="{} {}".format(prompt, positive_prompt),
        negative_prompt=negative_prompt,
        height=image_height,
        width=image_width,
        num_images_per_prompt=num_images_per_prompt,
        num_inference_steps=num_inference_steps,
        #guidance_scale=guidance_scale,
    ).images
        
    for j,image in enumerate(images):
        image.save("{}/sentence_{}_{}.png".format(utils.STORYBOARD_PATH, i+1, j+1))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
# Free GPU memory for upscaler
del pipeline
del images
torch.cuda.empty_cache()

In [1]:
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline

import torch

pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    revision="fp16",
    torch_dtype=torch.float16
)

text_encoder/model.safetensors not found


In [2]:
from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention(attention_op=MemoryEfficientAttentionFlashAttentionOp)
# Workaround for not accepting attention shape using VAE for Flash Attention
pipeline.vae.enable_xformers_memory_efficient_attention(attention_op=None)

In [3]:
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
 diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_ddim.DDIMScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscret

In [4]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [12]:
guidance_scale        = 0.7
num_inference_steps   = 20
num_images_per_prompt = 1
n = 5

for i,prompt in enumerate(scene_lines):
    for j in range(1,n+1):
        low_res_image = Image.open("outputs/robertson/scene_{}_{}.png".format(i+1,j)).convert("RGB")
        low_res_image = low_res_image.resize((256,256))

        image = pipeline(
            prompt="{} {}".format(prompt, positive_prompt),
            negative_prompt=negative_prompt,
            image=low_res_image,
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            #guidance_scale=guidance_scale,    
        ).images[0]

        image.save("outputs/robertson/scene_{}_{}_upscaled.png".format(i+1,j))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]